# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
clean_city_weather_df=pd.read_csv("City_Weather_Clean.csv")
clean_city_weather_df
clean_city_weather_df.describe()


,Unnamed: 0,Lattitude,Longitude,Temperature,Max Temperature,Humidity,Pressure,Cloud Cover,Wind Speed,Feels Like
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.00000,506.000000,506.000000
mean,276.664032,20.891146,20.299011,58.522431,59.308953,73.000000,1012.509881,57.44664,7.959842,57.215968
std,158.508274,33.484612,93.296601,19.804133,19.845622,18.282817,11.351142,40.49156,5.898052,23.476429
min,0.000000,-54.800000,-179.166700,-1.530000,-1.530000,16.000000,876.000000,0.00000,0.000000,-13.450000
25%,138.250000,-6.239300,-66.179825,43.800000,44.755000,62.000000,1010.000000,13.00000,3.315000,39.592500
50%,280.500000,26.352900,25.451700,60.845000,62.150000,77.000000,1013.000000,75.00000,6.400000,59.570000
75%,413.750000,48.508325,104.809250,76.507500,76.525000,86.000000,1018.000000,98.00000,11.500000,77.245000
max,548.000000,78.218600,179.316700,96.940000,96.940000,100.000000,1041.000000,100.00000,34.520000,100.200000


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
locations = clean_city_weather_df[["Lattitude", "Longitude"]]

In [4]:
humidity = clean_city_weather_df["Humidity"]
# Create a poverty Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# we pick cities with pleasant temps from 70 to 75 degrees
bucket_list_cities = clean_city_weather_df[(clean_city_weather_df["Max Temperature"] >= 70) &  (clean_city_weather_df["Max Temperature"] <= 75)]
bucket_list_cities.describe()
city_list=[]
hotel_list=[]
country_list=[]
hotel_lat=[]
hotel_lng=[]
city_lat=[]
city_lng=[]

# We filter it further to cities with comfortable humidites from 30 to 65
bucket_list_cities=bucket_list_cities[(bucket_list_cities["Humidity"] >= 30) &  (bucket_list_cities["Humidity"] <= 65)]
# We then filter it to sunny cities with low cloud cover
hotel_df=bucket_list_cities[(bucket_list_cities["Cloud Cover"] >= 0) &  (bucket_list_cities["Cloud Cover"] <= 65)]
# We now call the places API for each of the cities and store the first hotel name in a list
# we also store the lattitude and longitude for building a dataframe
for i, row in enumerate(hotel_df.values):
    # geocoordinates
    target_coordinates = f"{row[2]}, {row[3]}"
    target_search = "Hotel"
    target_radius = 5000
    target_type = "Hotel"
    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    # convert response to json
    places_data = response.json()
    #pprint(places_data)
    print(f' City {row[1]} -> Hotel : {places_data["results"][0]["name"]}')
    city_list.append(row[0])
    hotel_list.append(places_data["results"][0]["name"])
    country_list.append(places_data["results"][0]["name"])
    city_lat.append(row[2])
    city_lng.append(row[3])
    hotel_lat.append(places_data["results"][0]["geometry"]["location"]["lat"])
    hotel_lng.append(places_data["results"][0]["geometry"]["location"]["lng"])

hotel_results_df = pd.DataFrame({"City": city_list,
                                 "Hotel Name": hotel_list,
                                 "City Lattitude":city_lat,
                                 "City Longitude":city_lng,
                                 "Hotel Lattitude":hotel_lat,
                                 "Hotel Longitude":hotel_lng
                                })
hotel_results_df
    

 City avarua -> Hotel : The Islander Hotel
 City bela -> Hotel : Hotel Rajvilla
 City salta -> Hotel : Sheraton Salta Hotel
 City baringo -> Hotel : Ronella Cottages


,City,Hotel Name,City Lattitude,City Longitude,Hotel Lattitude,Hotel Longitude
0,122,The Islander Hotel,-21.2078,-159.7750,-21.199053,-159.795700
1,317,Hotel Rajvilla,25.9333,81.9833,25.921166,82.003705
2,503,Sheraton Salta Hotel,-24.7859,-65.4117,-24.785364,-65.398213
3,539,Ronella Cottages,0.4667,35.9667,0.450603,35.965725


In [6]:
# we create the locations list for gmaps layer api for hotel markers
hotel_locations = hotel_results_df[["Hotel Lattitude", "Hotel Longitude"]]

figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)
# Assign the marker layer to a variable
markers = gmaps.marker_layer(hotel_locations)
# Add the layer to the map
fig.add_layer(markers)
fig




Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

In [7]:
# we combine both the layers here
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))